## Endpoints

In [1]:
MARLOWE_RT_WEBSERVER_URL=http://192.168.0.12:13780

In [2]:
SIGNING_SERVICE_URL=http://192.168.0.12:13779

## Addresses and keys

### Party

In [3]:
cat party.testnet.address

addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs

In [4]:
cat party.skey

{
    "type": "PaymentSigningKeyShelley_ed25519",
    "description": "Payment Signing Key",
    "cborHex": "5820bb921c6af621b3336460e6df335d73d4d91012718240bb95dd0301fd9aa5b5e6"
}


### Counterparty

In [5]:
cat counterparty.testnet.address

addr_test1vrztl2rnzc47vt3llpzgh73lut3q3q3cer49qp9agsntaqgdxlyaw

## Contract

In [6]:
cat contract.json

{"when":[{"then":{"when":[{"then":{"when":[{"then":"close","case":{"notify_if":true}}],"timeout_continuation":"close","timeout":1685111280000},"case":{"for_choice":{"choice_owner":{"address":"addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs"},"choice_name":"Release"},"choose_between":[{"to":1,"from":1}]}}],"timeout_continuation":"close","timeout":1685024580000},"case":{"party":{"address":"addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs"},"of_token":{"token_name":"","currency_symbol":""},"into_account":{"address":"addr_test1vrztl2rnzc47vt3llpzgh73lut3q3q3cer49qp9agsntaqgdxlyaw"},"deposits":10000000}}],"timeout_continuation":"close","timeout":1684937880000}


In [7]:
json2yaml contract.json

timeout: 1684937880000
timeout_continuation: close
when:
- case:
    deposits: 10000000
    into_account:
      address: addr_test1vrztl2rnzc47vt3llpzgh73lut3q3q3cer49qp9agsntaqgdxlyaw
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      address: addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs
  then:
    timeout: 1685024580000
    timeout_continuation: close
    when:
    - case:
        choose_between:
        - from: 1
          to: 1
        for_choice:
          choice_name: Release
          choice_owner:
            address: addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs
      then:
        timeout: 1685111280000
        timeout_continuation: close
        when:
        - case:
            notify_if: true
          then: close


## Transaction 1. Create

### Prepare creation request

In [8]:
yaml2json << EOI > request-1.json
version: v1
contract: `cat contract.json`
roles: null
minUTxODeposit: 2000000
metadata: {}
tags: {}
EOI

cat request-1.json

{"contract":{"timeout":1684937880000,"timeout_continuation":"close","when":[{"case":{"deposits":10000000,"into_account":{"address":"addr_test1vrztl2rnzc47vt3llpzgh73lut3q3q3cer49qp9agsntaqgdxlyaw"},"of_token":{"currency_symbol":"","token_name":""},"party":{"address":"addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs"}},"then":{"timeout":1685024580000,"timeout_continuation":"close","when":[{"case":{"choose_between":[{"from":1,"to":1}],"for_choice":{"choice_name":"Release","choice_owner":{"address":"addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs"}}},"then":{"timeout":1685111280000,"timeout_continuation":"close","when":[{"case":{"notify_if":true},"then":"close"}]}}]}}]},"metadata":{},"minUTxODeposit":2000000,"roles":null,"tags":{},"version":"v1"}


### Call endpoint to build creation transaction

In [9]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H 'X-Change-Address: addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs' \
  -d @request-1.json \
  -o response-1.json \
  -sS

json2yaml response-1.json

links:
  contract: contracts/3c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c3329%231
resource:
  contractId: 3c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c3329#1
  txBody:
    cborHex: 86a40081825820012be528c8f2d28e3f7610533909a7b5a4cae5a4d02420da65d45049568bae04000182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0ba90d36a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a001e84800282005820a10bfb9106a6faf7ba77592d438bda4bd019cd33aa71fd954f0a9853a5bdd831021a0002d5e50b582067aa14f520459dae714839cd931c1627ac8965c515ab0002a4f59b42cb9145019fff81d8799fd8799f40ffd8799fa1d8799fd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffff1a001e8480a0a000ffd87c9f9fd8799fd8799fd8799fd87980d8799fd8799f581cc4bfa873162be62e3ff8448bfa3fe2e2088238c8ea5004bd4426be81ffd87a80ffffd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffd87a

### Find contract ID

In [10]:
CONTRACT_ID="$(jq -r '.resource.contractId' response-1.json)"
echo "CONTRACT_ID = $CONTRACT_ID"

CONTRACT_ID = 3c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c3329#1


### Find contract endpoint

In [11]:
CONTRACT_URL="$MARLOWE_RT_WEBSERVER_URL/1jq -r '.links.contract' response-1.json`"
echo "CONTRACT_URL = $CONTRACT_URL"

CONTRACT_URL = http://192.168.0.12:13780/contracts/3c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c3329%231


### Extract unsigned transaction body

In [12]:
jq '.resource.txBody' response-1.json > tx-1.unsigned

cat tx-1.unsigned

{
  "cborHex": "86a40081825820012be528c8f2d28e3f7610533909a7b5a4cae5a4d02420da65d45049568bae04000182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0ba90d36a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a001e84800282005820a10bfb9106a6faf7ba77592d438bda4bd019cd33aa71fd954f0a9853a5bdd831021a0002d5e50b582067aa14f520459dae714839cd931c1627ac8965c515ab0002a4f59b42cb9145019fff81d8799fd8799f40ffd8799fa1d8799fd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffff1a001e8480a0a000ffd87c9f9fd8799fd8799fd8799fd87980d8799fd8799f581cc4bfa873162be62e3ff8448bfa3fe2e2088238c8ea5004bd4426be81ffd87a80ffffd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffd87a9f1a00989680ffffd87c9f9fd8799fd87a9fd8799f4752656c65617365d8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffff9fd8799f0101ffffffd87c9f9fd8799fd87b9fd9

### Prepare signing request

In [13]:
yaml2json << EOI > signing-request-1.json
body: $(cat tx-1.unsigned)
paymentKeys: [$(cat party.skey)]
paymentExtendedKeys: []
EOI

cat signing-request-1.json

{"body":{"cborHex":"86a40081825820012be528c8f2d28e3f7610533909a7b5a4cae5a4d02420da65d45049568bae04000182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0ba90d36a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a001e84800282005820a10bfb9106a6faf7ba77592d438bda4bd019cd33aa71fd954f0a9853a5bdd831021a0002d5e50b582067aa14f520459dae714839cd931c1627ac8965c515ab0002a4f59b42cb9145019fff81d8799fd8799f40ffd8799fa1d8799fd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffff1a001e8480a0a000ffd87c9f9fd8799fd8799fd8799fd87980d8799fd8799f581cc4bfa873162be62e3ff8448bfa3fe2e2088238c8ea5004bd4426be81ffd87a80ffffd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffd87a9f1a00989680ffffd87c9f9fd8799fd87a9fd8799f4752656c65617365d8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffff9fd8799f0101ffffffd87c9f9fd8799fd87b

### Call endpoint to sign transaction

In [14]:
curl -sS \
  -H 'Content-Type: application/json' \
  -H 'Accept: application/json' \
  -d @signing-request-1.json \
  -o signing-response-1.json \
  $SIGNING_SERVICE_URL/sign
  
json2yaml signing-response-1.json

tx:
  cborHex: 84a40081825820012be528c8f2d28e3f7610533909a7b5a4cae5a4d02420da65d45049568bae04000182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0ba90d36a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a001e84800282005820a10bfb9106a6faf7ba77592d438bda4bd019cd33aa71fd954f0a9853a5bdd831021a0002d5e50b582067aa14f520459dae714839cd931c1627ac8965c515ab0002a4f59b42cb914501a20081825820ae56efbd10931edd0bcc4c1f929ab9c8d30fe15a58ea4df515aa281d1479a21c5840c145c64472c6349e34099eb39d77613a2d4ecc69bc18eaf0f7adec352992d1257f46e386d35b9314a396a0c6beff7f56481dac6b045b26ea917519a7722d2d0e0481d8799fd8799f40ffd8799fa1d8799fd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffff1a001e8480a0a000ffd87c9f9fd8799fd8799fd8799fd87980d8799fd8799f581cc4bfa873162be62e3ff8448bfa3fe2e2088238c8ea5004bd4426be81ffd87a80ffffd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4

### Extract the text envelope for the signed transaction

In [15]:
jq '.tx' signing-response-1.json > tx-1.signed

### Call the endpoint to submit the transaction

In [16]:
curl -sS \
  -X PUT \
  -H 'Content-Type: application/json' \
  -d @tx-1.signed \
  $CONTRACT_URL

### Poll to see if the transaction has been confirmed

In [17]:
curl -sS \
  -H 'Accept: application/json' \
  -o status-1.json \
  $CONTRACT_URL
  
jq -r '.resource.status' status-1.json

submitted


Wait a little while and then try again.

In [23]:
curl -sS \
  -H 'Accept: application/json' \
  -o status-1.json \
  $CONTRACT_URL
  
jq -r '.resource.status' status-1.json

confirmed


## Transaction 2. Deposit

### Prepare deposit request

In [27]:
yaml2json << EOI > request-2.json
version: v1
inputs:
- input_from_party:
    address: addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs
  that_deposits: 10000000
  of_token:
    currency_symbol: ''
    token_name: ''
  into_account:
    address: addr_test1vrztl2rnzc47vt3llpzgh73lut3q3q3cer49qp9agsntaqgdxlyaw
metadata: {}
tags: {}
EOI

cat request-2.json

{"inputs":[{"input_from_party":{"address":"addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs"},"into_account":{"address":"addr_test1vrztl2rnzc47vt3llpzgh73lut3q3q3cer49qp9agsntaqgdxlyaw"},"of_token":{"currency_symbol":"","token_name":""},"that_deposits":10000000}],"metadata":{},"tags":{},"version":"v1"}


### Call endpoint to build deposit transaction

In [28]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H 'X-Change-Address: addr_test1vp28qhdavped0rwr24ww8fmstr727a5ln7vrshlvq7cl3zgn0uaxs' \
  -d @request-2.json \
  -o response-2.json \
  -sS

json2yaml response-2.json

links:
  transaction: contracts/3c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c3329%231/transactions/e17bb66da0f01c21a43dfa1d2aa4ea876f41117f9b1e0724c7df93fb71517691
resource:
  contractId: 3c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c3329#1
  transactionId: e17bb66da0f01c21a43dfa1d2aa4ea876f41117f9b1e0724c7df93fb71517691
  txBody:
    cborHex: 86ab00828258203c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c3329008258203c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c3329010d818258203c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c33290012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0b076000a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a00b71b00028200582094c8b3ca0a4ee801ffc8511c0461099b07029d1a41c8ab560eeccace4a459a2310a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0b9b6b25111a000

### Find transaction endpoint

In [29]:
TRANSACTION_1_URL="$MARLOWE_RT_WEBSERVER_URL/`jq -r '.links.transaction' response-2.json`"
echo "TRANSACTION_1_URL = $TRANSACTION_1_URL"

TRANSACTION_1_URL = http://192.168.0.12:13780/contracts/3c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c3329%231/transactions/e17bb66da0f01c21a43dfa1d2aa4ea876f41117f9b1e0724c7df93fb71517691


### Extract unsigned transaction body

In [30]:
jq '.resource.txBody' response-2.json > tx-2.unsigned

cat tx-2.unsigned

{
  "cborHex": "86ab00828258203c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c3329008258203c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c3329010d818258203c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c33290012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0b076000a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a00b71b00028200582094c8b3ca0a4ee801ffc8511c0461099b07029d1a41c8ab560eeccace4a459a2310a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0b9b6b25111a000da211021a000916b6031a01be6418081a01bd3ed90e81581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f8890b58201ddcbb9a4f748fc94cd8e67a3a973757584973fd6052474528606c777fe995259fff82d8799fd8799f40ffd8799fa2d8799fd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffff1a001e8480d8799fd8799fd87980d8799fd879

### Prepare signing request

In [31]:
yaml2json << EOI > signing-request-2.json
body: $(cat tx-2.unsigned)
paymentKeys: [$(cat party.skey)]
paymentExtendedKeys: []
EOI

cat signing-request-2.json

{"body":{"cborHex":"86ab00828258203c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c3329008258203c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c3329010d818258203c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c33290012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0b076000a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a00b71b00028200582094c8b3ca0a4ee801ffc8511c0461099b07029d1a41c8ab560eeccace4a459a2310a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0b9b6b25111a000da211021a000916b6031a01be6418081a01bd3ed90e81581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f8890b58201ddcbb9a4f748fc94cd8e67a3a973757584973fd6052474528606c777fe995259fff82d8799fd8799f40ffd8799fa2d8799fd8799fd87980d8799fd8799f581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889ffd87a80ffffd8799f4040ffff1a001e8480d8799fd8799fd87980d8799f

### Call endpoint to sign transaction

In [32]:
curl -sS \
  -H 'Content-Type: application/json' \
  -H 'Accept: application/json' \
  -d @signing-request-2.json \
  -o signing-response-2.json \
  $SIGNING_SERVICE_URL/sign
  
json2yaml signing-response-2.json

tx:
  cborHex: 84ab00828258203c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c3329008258203c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c3329010d818258203c2a96dc4139f528ac9b1867cdf577c54c87ae398d4686b4a4ecc6391b3c33290012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010182a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0b076000a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a00b71b00028200582094c8b3ca0a4ee801ffc8511c0461099b07029d1a41c8ab560eeccace4a459a2310a200581d6054705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f889011a0b9b6b25111a000da211021a000916b6031a01be6418081a01bd3ed90e81581c54705dbd6072d78dc3555ce3a77058fcaf769f9f98385fec07b1f8890b58201ddcbb9a4f748fc94cd8e67a3a973757584973fd6052474528606c777fe99525a30081825820ae56efbd10931edd0bcc4c1f929ab9c8d30fe15a58ea4df515aa281d1479a21c584051b7026666ddfe8693d2d9b8091202d961039cc8bdaee37adc3ab1b7dee19ebf8ea75e61af5c7ec1aced295726d8ab6671c5bb1e5

### Extract the text envelope for the signed transaction

In [33]:
jq '.tx' signing-response-2.json > tx-2.signed

### Call the endpoint to submit the transaction

In [34]:
curl -sS \
  -X PUT \
  -H 'Content-Type: application/json' \
  -d @tx-2.signed \
  $TRANSACTION_1_URL

### Poll to see if the transaction has been confirmed

In [35]:
curl -sS \
  -H 'Accept: application/json' \
  -o status-2.json \
  $CONTRACT_URL
  
jq -r '.resource.status' status-2.json

submitted


Wait a little while and then try again.

In [36]:
curl -sS \
  -H 'Accept: application/json' \
  -o status-2.json \
  $CONTRACT_URL
  
jq -r '.resource.status' status-2.json

confirmed
